In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
import cellpy
from cellpy.utils import plotutils

In [ ]:
cell = cellpy.cell("../testdata/hdf5/20160805_test001_45_cc.h5", mass=0.8)

In [ ]:
plotutils.raw_plot(cell, ("Current", "current (A)"))

In [ ]:
plotutils.raw_plot(cell)

In [ ]:
# Another plotting option
def raw_plot2():
    # using holoviews (with bokeh backend)
    import numpy as np
    import holoviews as hv
    hv.extension("bokeh", logo=False)

    width = 800

    # full experiment
    all_cycs = hv.Curve(
        (cell.get_timestamp(), cell.get_voltage()), 
        ('t', 'time (sec)'), 
        ('v', 'voltage (v)'), 
        label="voltage-time"
    ).opts(
        width=width,
    )

    # cycle number labels
    cycle_label_df = cell.dataset.step_table.drop_duplicates("cycle")
    cycle_labels = hv.Labels((cycle_label_df.test_time_first, cycle_label_df.voltage_first, cycle_label_df.cycle))

    # pr cycle
    cycs_dict = dict()
    for c in cell.get_cycle_numbers():
        t = cell.get_timestamp(cycle=c)
        t = t - np.amin(t.values)  # setting first point to t=0
        curve = hv.Curve((t, cell.get_voltage(cycle=c)), ("time", "time (seconds)"), ("voltage", "voltage (v vs. Li/Li+)"))
        cycs_dict[c] = curve
    hmap = hv.HoloMap(cycs_dict,kdims=["cycle"]).opts(width=width)

    return ((all_cycs * cycle_labels) + hmap).cols(1)

In [ ]:
layout = raw_plot2()
layout

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure

In [ ]:
def _add_step_info_cols(df, table, cycles=None, steps=None, h_cycle=None, h_step=None):
    
    if h_cycle is None:
        h_cycle = "Cycle_Index"  # edit
    if h_step is None:
        h_step = "Step_Index"  # edit
        
    col_name_mapper = {
        "cycle": h_cycle,
        "step": h_step,
    }

    df = df.merge(
        table.rename(columns=col_name_mapper),
        on=('Cycle_Index', 'Step_Index'),
        how='left'
    )

    return df

# Another plotting option
def cycle_plot(cell, cycle=None, step=None, title=None, points=False, x=None, y=None,
               info_level=0,
               h_cycle=None, h_step=None,
               show_it=False, label_cycles=True, label_steps=False, **kwargs):
    
    """Plot raw data with annotations.
    
    This function uses Bokeh for plotting and is intended for use in 
    Jupyter Notebooks.
    """
    import numpy as np
    
    from bokeh.io import output_notebook, show
    from bokeh.layouts import row, column
    from bokeh.models import ColumnDataSource, LabelSet
    from bokeh.models import HoverTool
    from bokeh.models.annotations import Span
    from bokeh.models.widgets import Slider, TextInput
    from bokeh.plotting import figure
    
    output_notebook(hide_banner=True)
    
    if points: 
        if cycle is None or (len(cycle) > 1):
            print("Plotting points only allowed when plotting one single cycle.")
            print("Turning points off.")
            points = False
    
    if h_cycle is None:
        h_cycle = "Cycle_Index"  # edit
    if h_step is None:
        h_step = "Step_Index"  # edit
    
    if x is None:
        x = "Test_Time"  # edit
    if y is None:
        y = "Voltage"  # edit
        
    if isinstance(x, tuple):
        x, x_label = x
    else:
        x_label = x
        
    if isinstance(y, tuple):
        y, y_label = y
    else:
        y_label = y
    
    t_x = x  # used in generating title - replace with a selector
    t_y = y  # used in generating title - replace with a selector

    if title is None:
        title = f"{t_y} vs. {t_x}"

    cols = [x, y]
    cols.extend([h_cycle, h_step])
    
    df = cell.dataset.dfdata.loc[:, cols]

    if cycle is not None:
        if not isinstance(cycle, (list, tuple)):
            cycle = [cycle]
        
        _df = df.loc[df[h_cycle].isin(cycle), :]
        if len(cycle) < 5:
            title += f" [c:{cycle}]"
        else:
            title += f" [c:{cycle[0]}..{cycle[-1]}]"
        if _df.empty:
            print(f"EMPTY (available cycles: {df[h_step].unique()})")
            return
        else:
            df = _df

    cycle = df[h_cycle].unique()
        
    if step is not None:
        if not isinstance(step, (list, tuple)):
            step = [step]
            
        _df = df.loc[df[h_step].isin(step), :]
        if len(step) < 5:
            title += f" (s:{step})"
        else:
            title += f" [s:{step[0]}..{step[-1]}]"
        if _df.empty:
            print(f"EMPTY (available steps: {df[h_step].unique()})")
            return
        else:
            df = _df
        
    x_min, x_max = df[x].min(), df[x].max()
    y_min, y_max = df[y].min(), df[y].max()
    
    if info_level > 0:
        table = cell.dataset.step_table
        df = _add_step_info_cols(df, table, cycle, step)
    
    source = ColumnDataSource(df)

    plot = figure(
        title=title,
        tools="pan,reset,save,wheel_zoom,box_zoom,undo,redo",
        x_range=[x_min, x_max], y_range=[y_min, y_max],
        **kwargs,
    )
    
    plot.line(
        x, y, source=source, 
        line_width=3, line_alpha=0.6
    )
    
    # labelling cycles
    if label_cycles:
        cycle_line_positions = [df.loc[df[h_cycle]==c, x].min() for c in cycle]
        cycle_line_positions.append(df.loc[df[h_cycle]==cycle[-1], x].max())
        for m in cycle_line_positions:
            _s = Span(location=m, dimension='height', line_color="red", line_width=3,
                     line_alpha=0.5)
            plot.add_layout(_s)

        s_y_pos = y_min + 0.9 * (y_max - y_min)
        s_x = []
        s_y = []
        s_l = []

        for s in cycle:
            s_x_min = df.loc[df[h_cycle]==s, x].min()
            s_x_max = df.loc[df[h_cycle]==s, x].max()
            s_x_pos = (s_x_min + s_x_max)/2
            s_x.append(s_x_pos)
            s_y.append(s_y_pos)
            s_l.append(f"c{s}")

        c_labels = ColumnDataSource(data={
            x: s_x,
            y: s_y,
            'names': s_l

        })

        c_labels = LabelSet(x=x, y=y, text='names', level='glyph',
                  source=c_labels, render_mode='canvas', text_color="red", text_alpha=0.7)

        plot.add_layout(c_labels)    
    
    # labelling steps
    if label_steps:
        for c in cycle:
            step = df.loc[df[h_cycle]==c, h_step].unique()       
            step_line_positions = [df.loc[(df[h_step]==s) & (df[h_cycle]==c), x].min() for s in step[0:]]
            for m in step_line_positions:
                _s = Span(location=m, dimension='height', line_color="olive", line_width=3,
                         line_alpha=0.1)
                plot.add_layout(_s)
            
            #s_y_pos = y_min + 0.8 * (y_max - y_min)
            s_x = []
            s_y = []
            s_l = []

            for s in step:
                s_x_min = df.loc[(df[h_step]==s) & (df[h_cycle]==c), x].min()
                s_x_max = df.loc[(df[h_step]==s) & (df[h_cycle]==c), x].max()
                s_x_pos = s_x_min
                
                s_y_min = df.loc[(df[h_step]==s) & (df[h_cycle]==c), y].min()
                s_y_max = df.loc[(df[h_step]==s) & (df[h_cycle]==c), y].max()
                s_y_pos = (s_y_max + s_y_min)/2
                
                s_x.append(s_x_pos)
                s_y.append(s_y_pos)
                s_l.append(f"s{s}")

            s_labels = ColumnDataSource(data={
                x: s_x,
                y: s_y,
                'names': s_l

            })

            s_labels = LabelSet(x=x, y=y, text='names', level='glyph',
                      source=s_labels, render_mode='canvas', text_color="olive", text_alpha=0.3)

            plot.add_layout(s_labels)
            
    hover = HoverTool()
    if info_level==0:
        hover.tooltips = [
            (x, "$x{0.2f}"),
            (y, "$y"),
            ("cycle", f"@{h_cycle}"),
            ("step", f"@{h_step}"),
        ]
    elif info_level==1:
        # insert C-rates etc here
        hover.tooltips = [
            (f"(x,y)", "($x{0.2f} $y"),
            ("cycle", f"@{h_cycle}"),
            ("step", f"@{h_step}"),
            ("step_type", "@type"),
            ("rate", "@rate_avr{0.2f}")
        ]
        
    elif info_level==2:
        hover.tooltips = [
            (x, "$x{0.2f}"),
            (y, "$y"),
            ("cycle", f"@{h_cycle}"),
            ("step", f"@{h_step}"),
            ("step_type", "@type"),
            ("rate (C)", "@rate_avr{0.2f}"),
            ("dv (%)", "@voltage_delta{0.2f}"),
            ("I-max (A)", "@current_max"),
            ("I-min (A)", "@current_min"),
            ("dCharge (%)", "@charge_delta{0.2f}"),
            ("dDischarge (%)", "@discharge_delta{0.2f}"),
        ]
         
    hover.mode = 'vline'
    plot.add_tools(hover)
    
    plot.xaxis.axis_label = x_label
    plot.yaxis.axis_label = y_label
        
    if points:
        plot.scatter(
            x, y, source=source, 
            alpha=0.3
        )
        
    if show_it:
        show(plot)
    
    return plot


fig = cycle_plot(
    cell, 
    cycle=[3, 4, 5],
    info_level=2,
    height=400, 
    width=1200,
    show_it=True, 
    label_steps=True,
    label_cycles=True,
)